In [65]:
from pymatgen.analysis.adsorption import *
from pymatgen.core.surface import SlabGenerator, Structure
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.core.structure import Structure
from pymatgen.ext.matproj import MPRester
from pymatgen.io.vasp.inputs import Poscar
from pymatgen.core import Molecule
from pymatgen.io.cif import CifParser, CifWriter
from pymatgen.io.vasp.outputs import Vasprun
from pymatgen.electronic_structure.plotter import BSDOSPlotter

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

from ase.optimize import BFGS
from ase.visualize.plot import plot_atoms
from ase import Atoms
from ase.io import write
from ase.constraints import FixAtoms

from fairchem.core.models.model_registry import model_name_to_local_file
from fairchem.core.common.relaxation.ase_utils import OCPCalculator

import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap, Normalize
from matplotlib import rcParams

import seaborn as sns

import glob

import re

import pandas as pd

#命名方式举例：Ni5P4_mp-1920_111_slabnum_positionnum(_opt).cif

In [44]:
#一级函数(不直接调用)

# 获取一种晶体类型的最小能量
def select(sel_atom):
    file_pattern = f'{sel_atom}*.csv'  # 请确保这个模式匹配你的文件名
    file_list = glob.glob(file_pattern)  # 依次读取每个文件
    Selected_df = []
    
    for file_name in file_list:  # 使用pandas读取CSV文件
        df = pd.read_csv(file_name)  # 这里可以对每个dataframe进行操作，例如打印或者处理数据
        index = -1
        min_abs_energy = 999
        
        for i in range(len(df)):
            energy = abs(df['energy'][i])
            if energy <= min_abs_energy and energy >= 0.1:  # 添加能量值不能小于0.1的条件
                index = i
                min_abs_energy = energy
        
        if index > -1:
            Selected_df.append(df.iloc[index])

    index = -1
    min_abs_energy = 999
    
    for i in range(len(Selected_df)):
        energy = abs(Selected_df[i].energy)
        if energy <= min_abs_energy and energy >= 0.1:  # 添加能量值不能小于0.1的条件
            index = i
            min_abs_energy = energy
    
    if index > -1:
        return Selected_df[index]


#读取cif文件，并存储为POSCAR文件
def convert_cif_to_poscar(cif_filename, poscar_filename):
    """
    将 CIF 文件转换为 POSCAR 格式。

    参数:
    cif_filename (str): CIF 文件名
    poscar_filename (str): POSCAR 文件名

    返回:
    None
    """
    # 读取 CIF 文件
    structure = Structure.from_file(cif_filename)
    
    # 将结构保存为 POSCAR 格式
    poscar = Poscar(structure)
    poscar.write_file(poscar_filename)
    print(f"文件已保存为 {poscar_filename}")

#从Materials Project下载晶体结构并存储为cif文件,同时也存储一份dataframe文件
def get_structure_from_MP(Atoms_contains, filename, API_KEY):

    mpr = MPRester(API_KEY)

    # 使用 MPRester 对象连接到 Materials Project 数据库
    with MPRester(API_KEY) as m:
        # 查询特定的合金材料，例如 "AlCu"
        criteria = {"elements": {"$in": Atoms_contains, "$all": Atoms_contains}, "nelements": len(Atoms_contains)}
        properties = ["material_id", "pretty_formula", "structure"]

        # 获取数据
        results = m.query(criteria, properties)

        # 打印结果
    for result in results:
        material_id = result["material_id"]
        formula = result["pretty_formula"]
        structure = result["structure"]
        cif_writer = CifWriter(structure)
        cif_writer.write_file(f"{formula}_{material_id}.cif")

    #存储数据
    data = mpr.query(criteria={"elements": {"$in": Atoms_contains,"$all":Atoms_contains},'nelements' :len(Atoms_contains)}, properties=["material_id", "pretty_formula", "structure", "formation_energy_per_atom",  "e_above_hull","band_gap"])
    df = pd.concat([pd.DataFrame(data), pd.DataFrame(pd.Series(data[0]['structure'].as_dict()))], axis=1)
    df = df[df['e_above_hull']<0.1]

    #存储NiP的数据
    df.to_csv(filename, index=False)

#读取分子的cif文件，根据给定的米勒指数和层数和真空层生成第i个晶体的切片的ads_structs
def get_ads_structs(filename, hkl, layers, vacuum, slab_index, center_slab=True):
    # 读取分子的cif文件
    structure = Structure.from_file(filename+'.cif')
    #生成切片名称模板
    slab_name = f"{filename}_{hkl[0]}{hkl[1]}{hkl[2]}"
    # 生成吸附位点
    slab, attach_name = get_slab_cif(structure, hkl, layers, vacuum, slab_index, slab_name, center_slab)
    asf = AdsorbateSiteFinder(slab[slab_index])
    #将氢原子吸附到吸附点上
    adsorbate = Molecule("H", [[0,0,0]])
    #生成吸附结构
    ads_structs = asf.generate_adsorption_structures(adsorbate, repeat=[1, 1, 1])
    #生成吸附位点名称模板
    return ads_structs, attach_name

#将晶体进行切割并生成可能的吸附位点，将氢原子吸附上去后，存储为cif文件
def get_slab(structure, hkl, layers, vacuum, center_slab=True):
    structure = SpacegroupAnalyzer(structure).get_conventional_standard_structure()
    slab = SlabGenerator(structure, hkl, layers, vacuum, center_slab)
    slab = slab.get_slabs()
    return slab

def write_cif(slab, slabnum, filename):
    slab = slab[slabnum]
    cif_writer = CifWriter(slab)
    slab_num_name = f"{filename}_{slabnum}"
    cif_writer.write_file(f"{filename}_{slabnum}.cif")
    return slab_num_name

def get_slab_cif(structure, hkl, layers, vacuum, slab_num, filename, center_slab=True):
    slab = get_slab(structure, hkl, layers, vacuum, center_slab)
    slab_num_name = write_cif(slab, slab_num, filename)
    return slab, slab_num_name

#获取一个米勒指数的所有晶胞切片的数量
def get_slab_num(filename, hkl, layers, vacuum, center_slab=True):
    structure = Structure.from_file(filename+'.cif')
    slab = get_slab(structure, hkl, layers, vacuum, center_slab)
    return len(slab)

#获取一个米勒指数的所有晶胞切片(不写入文件)
def get_slab_origin(filename, hkl, layers, vacuum, center_slab=True):
    structure = Structure.from_file(filename+'.cif')
    slab = get_slab(structure, hkl, layers, vacuum, center_slab)
    return slab

#读取cif文件，生成structure对象,并转换为Atoms对象
def atom_from_file(filename):
    #读取晶胞参数
    sites = Structure.from_file(filename)
    
    # 定义晶胞参数
    cell = sites.lattice.matrix
    # 定义原子信息
    symbols = []
    positions = []

    # 从sites变量中提取原子信息
    for site in sites:
        element = site.species_string # 原子元素
        frac_coords = site.frac_coords # 原子的分数坐标
        symbols.append(element)
        positions.append([frac_coords[0], frac_coords[1], frac_coords[2]])

    #创建Atoms对象
    ni5p4 = Atoms(symbols=symbols, scaled_positions=positions, cell=cell, pbc=True)

    #打印Atoms对象信息
    #print(ni5p4)
    return ni5p4

#将Structure对象转换为Atoms对象
def Structure_to_Atom(structure):
    #读取晶胞参数
    sites = structure
    
    # 定义晶胞参数
    cell = sites.lattice.matrix
    # 定义原子信息
    symbols = []
    positions = []

    # 从sites变量中提取原子信息
    for site in sites:
        element = site.species_string # 原子元素
        frac_coords = site.frac_coords # 原子的分数坐标
        symbols.append(element)
        positions.append([frac_coords[0], frac_coords[1], frac_coords[2]])

    #创建Atoms对象
    Atom = Atoms(symbols=symbols, scaled_positions=positions, cell=cell, pbc=True)

    #打印Atoms对象信息
    return Atom

#训练模型，优化吸附结构并返回该结构
def Atoms_training(atoms, model_name):
    # 加载模型
    checkpoint_path = model_name_to_local_file(model_name, local_cache='/tmp/fairchem_checkpoints/')
    # 训练模型
    calc = OCPCalculator(checkpoint_path=checkpoint_path, cpu=False)
    atoms.set_constraint(FixAtoms(mask=[atom.tag >1 for atom in atoms]))
    atoms.set_calculator(calc)
    # 优化结构
    opt = BFGS(atoms)
    opt.run(fmax=0.05, steps=400)
    # 返回能量值
    return atoms

#将某一方向所有的氢原子位点分别存储为cif文件
# 将 ads_structs 存储为 CIF 文件，返回这个方向的氢原子位点的个数
def save_ads_structs(ads_structs, prefix):
    num_structs = 0
    for i, struct in enumerate(ads_structs):
        filename = f"{prefix}_{i}.cif"
        struct.to(filename=filename)
        print(f"Saved {filename}")
        num_structs += 1
    return num_structs

#读取某一方向所有的氢原子位点的cif文件并存储为Structure对象
def read_ads_structs(prefix, num_structs):
    ads_structs = []
    for i in range(num_structs):
        filename = f"{prefix}_{i}.cif"
        struct = Structure.from_file(filename)
        ads_structs.append(struct)
    return ads_structs

#将计算得到的晶胞编号，能量值，米勒指数，层数，真空层，晶胞切片的索引存储为dataframe文件
def save_csv(filename, node_number, energy, hkl, layer, vacuum, slab_index):
    data = {"slab_name":filename,"node_number": node_number, "energy": energy, "hkl": hkl, "layer": layer, "vacuum": vacuum, "slab_index": slab_index}
    df = pd.DataFrame(data)
    df.to_csv(f"{filename}.csv")
    return df.head(1)

#将晶体切片的所有信息存储为cif文件，并返回一个dataframe
def total(filename,hkl,layer,vacuum,slab_index):
    ads_structs, attach_name= get_ads_structs(filename, hkl, layer, vacuum, slab_index, center_slab=True)
    #存储与读取晶胞切片
    # 将 ads_structs 存储为 CIF 文件
    node_number = save_ads_structs(ads_structs, attach_name)
    # 读取存储的 CIF 文件
    all_positions = read_ads_structs(attach_name, node_number)
    list_of_energy = []
    #存储优化后的晶胞的所有信息
    list_of_atoms = []
    for i in range(0,node_number):
        selected_position = Structure_to_Atom(all_positions[i])
        selected_position_opted = Atoms_training(selected_position, "EquiformerV2-83M-S2EF-OC20-2M")
        opt_name = f"{attach_name}_{i}_opt"
        #使用ase的write函数(针对Atoms类)将氢原子吸附后的晶胞存储为cif文件
        write(opt_name+'.cif', selected_position_opted)
        list_of_energy.append(selected_position_opted.get_total_energy())
        #将计算得到的晶胞编号，能量值，米勒指数，层数，真空层，晶胞切片的索引存储为一个dataframe
        list_of_atoms.append(save_csv(opt_name, i, list_of_energy[i], hkl, layer, vacuum, slab_index))
    return list_of_atoms

#处理数据并将其存储为 CSV 文件
def process_data(filename, hkl, layer, vacuum, slab_index):
    """
    处理数据并将其存储为 CSV 文件。

    参数:
    filename (str): 文件名
    hkl (list): hkl 值
    layer (int): 层数
    vacuum (int): 真空层厚度
    slab_index (int): 板层索引

    返回:
    None
    """
    # 调用 total 函数
    data_opted = total(filename, hkl, layer, vacuum, slab_index)
    
    # 打印第一个 DataFrame 以检查数据
    print(data_opted[0])
    
    # 将 data_opted 转化为一个 DataFrame 并重新生成索引
    df = pd.concat(data_opted)
    df = df.reset_index(drop=True)
    
    # 打印 DataFrame 以检查数据
    print(df)
    
    # 将 DataFrame 存储为 CSV 文件
    csv_filename = f"{filename}_{hkl[0]}{hkl[1]}{hkl[2]}_{layer}_{vacuum}_{slab_index}.csv"
    df.to_csv(csv_filename, index=False)
    print(f"Data saved to {csv_filename}")

#绘制给定VASP晶体文件的能带图
def plot_band_structure(filename):
    # 读取 VASP 输出文件
    bs_vasprun = Vasprun(filename,parse_projected_eigen=True)
    bs_data = bs_vasprun.get_band_structure(line_mode=True)

    dos_vasprun=Vasprun(filename)
    dos_data=dos_vasprun.complete_dos

    banddos_fig = BSDOSPlotter(bs_projection=None, dos_projection=None, vb_energy_range=5, fixed_cb_energy=5)
    banddos_fig.get_plot(bs=bs_data, dos=dos_data)
    plt.savefig('banddos_fig.png', img_format='png')

In [ ]:
'''
structure:
类型: pymatgen.core.structure.Structure
功能: 代表一个晶体结构。它包含了晶体的原子位置、晶格参数和其他相关信息。你可以从文件（如 CIF 文件）中读取结构数据，并对其进行各种操作，如切割晶体、添加缺陷等。

slab:
类型: pymatgen.core.structure.Structure 或 pymatgen.core.structure.Structure 的列表
功能: 代表一个从晶体结构中切割出来的表面（slab）。在代码中，slab 是通过 get_slab_cif 函数从 structure 中切割出来的。它通常用于表面科学研究，如吸附、催化等。

adsorbate:
类型: pymatgen.core.structure.Molecule
功能: 代表一个分子或原子团簇，通常用于吸附研究。在代码中，adsorbate 是一个氢原子分子，用于模拟氢原子在表面上的吸附。

ads_structs:
类型: list of pymatgen.core.structure.Structure
功能: 代表一组吸附结构。每个结构都是一个包含吸附物（如氢原子）和基底（如 slab）的组合。在代码中，ads_structs 是通过 generate_adsorption_structures 函数生成的，表示氢原子在不同吸附位点上的结构。

    # 替换为你的 Materials Project API 密钥
    API_KEY = "7cXC8xULfZusdoLODDA"

'''


In [9]:
#二级函数（直接调用）

#读取固定元素组成的所有晶体，调用函数并存储为dataframe文件
def get_structs_from_MP(Atoms_contains, API_KEY):
    filename_elements = ""
    for i in range(len(Atoms_contains)):
        filename_elements += Atoms_contains[i]
    get_structure_from_MP(Atoms_contains,filename_elements+".csv",API_KEY)
    df = pd.read_csv(filename_elements+".csv")
    return df, filename_elements

#读取dataframe文件，吸附氢原子，调用预训练模型优化并存储为cif文件
def process_all_data(df):
    for k in range(len(df)):
        filename = df['pretty_formula'][k]+'_'+df['material_id'][k]
        for i in range(7):    
            hkl = [[1,0,0],[0,1,0],[0,0,1],[1,1,0],[1,0,1],[0,1,1],[1,1,1]]
            layer = 6
            vacuum = 10
            slab_num =  get_slab_num(filename, hkl[i], layer, vacuum)
            for j in range(slab_num):
                slab_index = j
                process_data(filename, hkl[i], layer, vacuum, slab_index)

#将每种晶型的H吸附能最小的切面、位点数据存入csv文件
def find_min_energy(df, filename_elements):
    crystal_list = []
    sel_list = []

    for i in range(len(df)):
        sel_list.append(f"{df['pretty_formula'][i]}_{df['material_id'][i]}")

    for sel_atom in sel_list:
        if select(sel_atom) is not None:
            crystal_list.append(select(sel_atom))

    crystal_DF=pd.DataFrame(crystal_list)
    crystal_DF.to_csv(f"crystal_{filename_elements}.csv")
    return f"crystal_{filename_elements}"

#将筛选出来的晶体结构的cif文件转换为POSCAR文件
def selected_cif_to_poscar(all_crystal_names):
    df_cif = pd.read_csv(f"{all_crystal_names}.csv")
    for crystal_name in df_cif["slab_name"]:
        convert_cif_to_poscar(crystal_name+".cif", crystal_name+".POSCAR")

#从文件里读入筛选出来的晶体结构的cif文件
def read_in_csv(all_crystal_names):
    selected = pd.read_csv(f"{all_crystal_names}.csv")
    selected_name = selected['slab_name']
    name_ing = []
    for k in range(len(selected_name)):
        name_ing.append(selected_name[k].split('_'))
    return name_ing

#获取筛选出来的晶体结构未吸附H原子的切片，并存储为cif文件与POSCAR文件
def get_origin_slabs(df, name_ing):
    all_original_slabs_data = []
    original_file_name = []
    for k in range(len(df)):
        filename = df['pretty_formula'][k]+'_'+df['material_id'][k]
        for i in range(7):    
            hkl = [[1,0,0],[0,1,0],[0,0,1],[1,1,0],[1,0,1],[0,1,1],[1,1,1]]
            layer = 6
            vacuum = 10
            hkl_simple = ['100','010','001','110','101','011','111']
            for x in range(len(name_ing)):
                if name_ing[x][0] == df['pretty_formula'][k] and name_ing[x][1] == df['material_id'][k] and name_ing[x][2] == hkl_simple[i]:
                    slab =  get_slab_origin(filename, hkl[i], layer, vacuum)
                    original_file_name.append(f"{name_ing[x][0]}_{name_ing[x][1]}_{name_ing[x][2]}_{name_ing[x][3]}.cif")
                    all_original_slabs_data.append(slab[int(name_ing[x][3])])
                    cif_writer_ori = CifWriter(slab[int(name_ing[x][3])])
                    cif_writer_ori.write_file(f"{name_ing[x][0]}_{name_ing[x][1]}_{name_ing[x][2]}_{name_ing[x][3]}.cif")
                    convert_cif_to_poscar(f"{name_ing[x][0]}_{name_ing[x][1]}_{name_ing[x][2]}_{name_ing[x][3]}.cif",f"{name_ing[x][0]}_{name_ing[x][1]}_{name_ing[x][2]}_{name_ing[x][3]}.POSCAR")

#获取晶胞图片
def draw_slab_picture(df, name_ing):
    for k in range(len(df)):
        filename = df['pretty_formula'][k]+'_'+df['material_id'][k] 
        for i in range(7):    
            hkl = [[1,0,0],[0,1,0],[0,0,1],[1,1,0],[1,0,1],[0,1,1],[1,1,1]]
            layer = 5
            vacuum = 10
            hkl_simple = ['100','010','001','110','101','011','111']
            for x in range(len(name_ing)):
                if name_ing[x][0] == df['pretty_formula'][k] and name_ing[x][1] == df['material_id'][k] and name_ing[x][2] == hkl_simple[i]:
                    slab_index = int(name_ing[x][3])
                    ads_structs, attach_name = get_ads_structs(filename, hkl[i], layer, vacuum, slab_index, center_slab=True)
                    new_atoms = Atoms_training(Structure_to_Atom(ads_structs[int(name_ing[x][4])]), "GemNet-dT-S2EF-OC20-2M")
                    fig = plt.figure()
                    ax = fig.add_subplot(111)
                    plot_atoms(new_atoms, ax, rotation = '-45x,-30y')
                    plt.savefig(f"{name_ing[x][0]}_{name_ing[x][1]}_{name_ing[x][2]}_{name_ing[x][3]}_{name_ing[x][4]}.png", dpi=300)
                    plt.show()
                    plt.clf()

#显示切面上吸附位点的吸附能散点图
def draw_slab_points(name_ing, x):
    df = pd.read_csv(f"{name_ing[x][0]}_{name_ing[x][1]}_{name_ing[x][2]}_4_10_{name_ing[x][3]}.csv")
    energy = df['energy']

    cif_file = f"{name_ing[x][0]}_{name_ing[x][1]}_{name_ing[x][2]}_{name_ing[x][3]}*.cif"
    cif_list = glob.glob(cif_file)
    atom_data = []
    for cif_name in cif_list:
        parser = CifParser(cif_name)
        structure = parser.parse_structures()[0]

        target_element = "H"
        for site in structure:
            if site.specie.symbol == target_element:
                atom_info = {
                    "Element": site.specie.symbol,
                    "x": site.coords[0],
                    "y": site.coords[1],
                    "z": site.coords[2]
                }
                atom_data.append(atom_info)
    df2 = pd.DataFrame(atom_data)
    axis_x = df2['x']
    axis_y = df2['y']
    axis_z = df2['z']

    x = []
    y = []
    z = []
    e = []
    for i in range(len(axis_x)):
        x.append(axis_x[i])
        y.append(axis_y[i])
        z.append(axis_z[i])
        e.append(energy[i])

    colors = [(0, 0, 1), (1, 0, 0), (0, 0, 1)]  # 蓝色到红色再到蓝色
    cmap = LinearSegmentedColormap.from_list("custom_cmap", colors, N=256)

    vmin = -0.8
    vmax = 0.8
    vcenter = 0

    # 创建规范化对象
    norm = Normalize(vmin=vmin, vmax=vmax)

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # 绘制散点图
    scatter = ax.scatter(x, y, z, c=e, cmap=cmap,norm=norm,s=20)

    # 添加颜色条
    cbar = fig.colorbar(scatter)
    cbar.set_label('Energy')

    # 设置轴标签
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title(f'Attachment points of {name_ing[x][0]}_{name_ing[x][1]}_{name_ing[x][2]}_{name_ing[x][3]}')

    # 显示图表
    plt.show()
    plt.clf()

#绘制晶体切面随米勒指数变化的箱线图和直方图
def process_and_plot_data():
    # 设置Seaborn的主题和调色板
    sns.set_theme(style="whitegrid")
    palette = sns.color_palette("Set2")

    # 获取所有CSV文件的列表
    csv_files = glob.glob("Ni2P_mp-21167_*_opt.csv")

    # 读取所有CSV文件并合并成一个DataFrame，并提取米勒指数
    dataframes = []
    for file in csv_files:
        df = pd.read_csv(file)
        # 提取米勒指数
        miller_index = re.search(r'_(\d{3})_', file).group(1)
        df['miller_index'] = miller_index
        # 保留能量值绝对值最小的那一行
        df = df.loc[df['energy'].abs().idxmin()]
        dataframes.append(df)

    combined_df = pd.DataFrame(dataframes)

    # 按照米勒指数进行分组
    grouped = combined_df.groupby('miller_index')

    # 统计描述
    stats = grouped['energy'].describe()
    print(stats)

    # 绘制箱线图
    plt.figure(figsize=(15, 10))
    sns.boxplot(x='miller_index', y='energy', data=combined_df, palette=palette)
    plt.title('Box plots of energy values for different Miller indices of Ni2P', fontsize=16)
    plt.xlabel('Miller indices', fontsize=14)
    plt.ylabel('Hydrogen atom adsorption energy', fontsize=14)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.show()

    # 绘制直方图
    plt.figure(figsize=(15, 10))
    for name, group in grouped:
        sns.histplot(group['energy'], kde=True, label=f'Miller indices {name}', alpha=0.5, palette=palette)

    plt.title('Histograms of energy values for different Miller exponents of Ni2P', fontsize=16)
    plt.xlabel('Hydrogen atom adsorption energy', fontsize=14)
    plt.ylabel('Frequency', fontsize=14)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.legend(title='Miller indices', fontsize=12, title_fontsize=14)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.show()

In [10]:
#主函数示例
df, filename_elements = get_structs_from_MP(["Ni","P"], "7cXC8xULfZusdoLODDA")
process_all_data(df)
all_crysal_names = find_min_energy(df, filename_elements)
selected_cif_to_poscar(all_crysal_names)
name_ing = read_in_csv(all_crysal_names)
get_origin_slabs(df, name_ing)
draw_slab_picture(df, name_ing)
draw_slab_points(name_ing, 0)